# Chapter 10. 특성 선택을 사용한 차원 축소

## 10.0 소개
* 특성 선택 방식 : 필터, 래퍼, 임베디드

    * 필터(filter) : 통계 속성을 조사하여 가장 뛰어난 특성 선택
    * 래퍼(wrapper) : 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합을 찾는다.
    * 임베디드(embedded) : 알고리즘의 훈련 단계를 확장하거나 일부로 구성하여 가장 좋은 특성의 부분 조합 선택 

## 10.1 분산을 기준으로 수치 특성 선택하기
* 주어진 기준값보다 높은 분산을 가진 특성 선택

In [1]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

features = iris.data
target = iris.target

thresholder = VarianceThreshold(threshold=.5)

features_high_variance = thresholder.fit_transform(features)

# 기준값 보다 높은 특성 선택 후 그 특성을 확인
features_high_variance[0:3]

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

* 분산 기준 설정(variance thresholding, VT)
    * 분산이 높은 특성이 낮은 것 보다 더 효과적이거나 유용하지 않ㅎ다.
    * 각 특성의 분산 계산
    
$$Var(x) = {1 \over n} \sum_{i=1}^n (x_i - \mu)^2$$

In [2]:
# 분산 확인은 variances_ 속성
thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_std = scaler.fit_transform(features)

selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## 10.2 분산을 기준으로 이진 특성 선택하기
* 베르누이 확률 변수의 분산이 기준값 이상인 특성 선택

In [6]:
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0],
           [0, 1, 1],
           [0, 1, 0],
           [0, 1, 1],
           [1, 0, 0]]

thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

In [7]:
thresholder.variances_

array([0.16, 0.16, 0.24])

In [8]:
import numpy as np
np.var(featuers, axis=0)

array([0.16, 0.16, 0.24])

## 10.3 상관관계가 큰 특성 다루기

* 상관관계 행렬(correlation matrix)

In [9]:
import numpy as np
import pandas as pd

features = np.array([[1,1,1],
                    [2,2,0],
                    [3,3,1],
                    [4,4,0],
                   [5,5,1],
                    [6,6,0],
                    [7,7,1],
                    [8,7,0],
                    [9,7,1]])

dataframe = pd.DataFrame(features)

corr_matrix = dataframe.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [10]:
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [11]:
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [12]:
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [13]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [14]:
np.triu(np.ones((4,4)), k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [15]:
np.tril(np.ones((4,4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

## 10.4 분류 작업에 관련 없는 특성 삭제하기
* 범주형 특성이라면 각 특성과 타깃 벡터 사이의 카이제곱 통계 계산

In [16]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = load_iris()
features = iris.data
target = iris.target

features = features.astype(int)

chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [17]:
# F-값이 가장 높은 특성 두 개를 선택한다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [18]:
from sklearn.feature_selection import SelectPercentile

fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 3


$$\chi^2 = \sum_{i=1}^n {(O_i - E_i)^2 \over E_i}$$

In [19]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [20]:
observed = np.sum(features.reshape(3,50,4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [21]:
expected = features.sum(axis=0) / 3

In [22]:
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [23]:
np.sum((observed - expected) ** 2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [24]:
# 카이제곱 점수를 확인
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [25]:
total_mean = np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [26]:
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [27]:
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

In [28]:
ss_total = np.sum((features - total_mean) **2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [29]:
f = (ss_between/(3-1)) / ((ss_total - ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [30]:
fvalue_selector.scores_

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

* 회귀라면 f_regression 함수 사용

## 10.5 재귀적 특성 제거하기
* 사이킷런의 RFECV : 재귀적 특성 제거를 교차 검증으로 수행한다.

In [72]:
# 라이브러리를 임포트합니다.
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error",min_features_to_select=3)
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 , -1.13210747],
       [-1.07500204,  2.56148527,  0.46847025],
       [ 1.37940721, -1.77039484,  0.98059736],
       ...,
       [-0.80331656, -1.60648007, -0.40526417],
       [ 0.39508844, -1.34564911, -0.66344459],
       [-0.55383035,  0.82880112, -1.24820021]])

In [73]:
# 최선의 특성 개수
rfecv.n_features_

3

In [74]:
# 남은 특성을 확인한다. 불리언 마스크 확인
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [75]:
# 특성의 순위 확인: 최고(1)에서 최악(96)
rfecv.ranking_

array([96, 62, 72, 98, 18,  1, 39, 26, 14, 65, 56, 34, 13,  4, 83, 51, 15,
       46, 86, 92, 48, 16, 19, 42, 59,  8, 40, 66, 27, 61, 63, 33,  7, 10,
       88, 84,  5, 12, 54,  1,  9,  6, 38, 58, 79, 35, 45, 82, 20, 11, 53,
       95, 30, 97, 23,  3, 37, 70, 93, 28, 22, 67, 50, 94, 47, 55, 78, 85,
       89, 31, 91, 32, 29, 76, 44, 52, 80, 69, 57, 74, 36, 24, 25, 71, 41,
        1, 68, 81, 77, 49, 21, 64, 60, 73, 17, 75,  2, 90, 43, 87])

* 타깃, 특성 행렬
    * 데이터 훈련 셋과 테스트 셋 두 개의 그룹 나누기
    * 훈련 세트로 모델 훈련
    * 테스트 셋의 타깃을 모르고, 테스트 셋의 특성을 모델에 적용하여 테스트 셋의 타깃값 예측
    * 예측한 타깃값과 실제 타깃값 비교하여 모델 평가

In [76]:
# 교차검증을 쓰지 않는 재귀적 특성 제거 법인 RFE 클래스 제공
# 남길 최소 특성의 개수가 입력 특성의 절반이다.
# 3개의 특성이 남도록 RFE 훈련

from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)
rfe.fit(features, target)
rfe.transform(features)

array([[ 0.00850799,  0.7031277 , -1.13210747],
       [-1.07500204,  2.56148527,  0.46847025],
       [ 1.37940721, -1.77039484,  0.98059736],
       ...,
       [-0.80331656, -1.60648007, -0.40526417],
       [ 0.39508844, -1.34564911, -0.66344459],
       [-0.55383035,  0.82880112, -1.24820021]])

In [77]:
# rfe 특성과 rfecv 특성이 동일한지 확인하고자 불리언 마스크 비교한다.
# 넘파이 all 함수: 모든 원소가 TRUE인지 검사한다.
np.all(rfe.support_ == rfecv.support_)

True